In [ ]:
import google.generativeai as genai
import typing_extensions as typing
from typing import List, Dict, TypedDict
import json
import re
import os
from janome.tokenizer import Tokenizer
from tqdm import tqdm
import subprocess
import srt
from datetime import timedelta

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

class SubtitleEntry(TypedDict):
    index: int
    start: str
    end: str
    text: str

class TranslationResponse(TypedDict):
    index: int
    translation: str

translation_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "index": {"type": "integer"},
            "translation": {"type": "string"}
        },
        "required": ["index", "translation"]
    }
}
 
def get_srt_files():
    return [f for f in os.listdir(".") if f.endswith(".srt")]

def select_srt_file():
    srt_files = get_srt_files()
    if not srt_files:
        print("No .srt files found in the current directory.")
        return None

    print("Available .srt files:")
    for i, file in enumerate(srt_files):
        print(f"{i+1}. {file}")

    while True:
        try:
            choice = int(input("Enter the number of the file you want to translate: "))
            if 1 <= choice <= len(srt_files):
                return srt_files[choice - 1]
            else:
                print(
                    "Invalid choice. Please enter a number between 1 and",
                    len(srt_files),
                )
        except ValueError:
            print("Invalid input. Please enter a number.")


def create_video_with_subtitles(video_file, subtitle_file, output_video_file):
    """字幕付きのビデオを作成します。

    Args:
        video_file (str): 入力ビデオファイルのパス。
        subtitle_file (str): 字幕ファイルのパス。
        output_video_file (str): 出力ビデオファイルのパス。

    Returns:
        None
    """
    process = subprocess.Popen(
        [
            "ffmpeg",
            "-hwaccel",
            "auto",
            "-i",
            f"{video_file}",
            "-vf",
            f"subtitles={subtitle_file}:force_style='FontName=Helvetica,FontSize=11'",
            "-c:v",
            "h264_amf",
            "-c:a",
            "copy",
            "-progress",
            "-",
            f"{output_video_file}",
        ],
        stderr=subprocess.PIPE,
        universal_newlines=True,
    )

    while True:
        line = process.stderr.readline()
        if line == "" and process.poll() is not None:
            break
        if "frame=" in line:
            print(line.strip())

    if process.returncode != 0:
        print(f"Error creating video: {process.returncode}")

class SRTTranslator:
    def __init__(self, api_key: str, batch_size: int = 20):
        genai.configure(api_key=GOOGLE_API_KEY)
        self.model = genai.GenerativeModel(
            "gemini-1.5-flash-002",
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": translation_schema
            }
        )
        self.batch_size = batch_size

    def timedelta_to_str(self, td: timedelta) -> str:
        hours = td.seconds // 3600
        minutes = (td.seconds % 3600) // 60
        seconds = td.seconds % 60
        milliseconds = td.microseconds // 1000
        return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

    def srt_to_json(self, file_path: str) -> List[SubtitleEntry]:
        subtitles = list(srt.parse(open(file_path, 'r', encoding='utf-8').read()))
        return [
            {
                "index": sub.index,
                "start": self.timedelta_to_str(sub.start),
                "end": self.timedelta_to_str(sub.end),
                "text": sub.content
            }
            for sub in subtitles
        ]

    def create_translation_request(self, subtitles: List[SubtitleEntry]) -> str:
        """翻訳リクエスト用のJSONを作成"""
        # 翻訳に必要な情報（indexとtext）のみを含むデータを作成
        simplified_subtitles = [
            {
                "index": sub["index"],
                "text": sub["text"]
            }
            for sub in subtitles
        ]
        
        request_data = {
            "task": "translate",
            "source_language": "English",
            "target_language": "Japanese",
            "subtitles": simplified_subtitles
        }
        
        # プロンプトの指示をJSON内に含める
        instruction = {
            "instructions": [
                "Translate each subtitle from English to natural Japanese",
                "Maintain the original meaning while making it sound natural",
                "Keep the original subtitle index numbers in your response",
                "Return the translations in the specified JSON format"
            ]
        }
        
        request_data.update(instruction)
        return json.dumps(request_data, ensure_ascii=False, indent=2)

    def translate_batch(self, subtitles: List[SubtitleEntry]) -> List[Dict]:
        """字幕バッチを翻訳"""
        json_request = self.create_translation_request(subtitles)
        
        print("\n=== 送信データ（JSON） ===")
        print(json_request)
        print("================\n")
        
        # JSONリクエストを送信
        response = self.model.generate_content(json_request)
        
        print("\n=== 受信データ（JSON） ===")
        print(response.text)
        print("================\n")
        
        try:
            translations = json.loads(response.text)
            
            # 翻訳結果を元の字幕データと結合
            translated_subtitles = []
            for sub in subtitles:
                translation = next(
                    (t['translation'] for t in translations if t['index'] == sub['index']),
                    sub['text']  # 翻訳が見つからない場合は元のテキストを使用
                )
                translated_sub = sub.copy()
                translated_sub['text'] = translation
                translated_subtitles.append(translated_sub)
                
            return translated_subtitles
            
        except json.JSONDecodeError as e:
            print(f"JSON解析エラー: {e}")
            return subtitles  # エラーの場合は元の字幕を返す
        
    def json_to_srt(self, subtitles: List[Dict], output_path: str):
        """JSON形式の字幕データをSRTファイルに変換"""
        srt_subtitles = []
        
        for sub in subtitles:
            # SRT形式の時間文字列をTimedeltaに変換
            start_parts = sub['start'].replace(',', ':').split(':')
            end_parts = sub['end'].replace(',', ':').split(':')
            
            start = timedelta(
                hours=int(start_parts[0]),
                minutes=int(start_parts[1]),
                seconds=int(start_parts[2]),
                milliseconds=int(start_parts[3])
            )
            
            end = timedelta(
                hours=int(end_parts[0]),
                minutes=int(end_parts[1]),
                seconds=int(end_parts[2]),
                milliseconds=int(end_parts[3])
            )
            
            srt_subtitles.append(
                srt.Subtitle(
                    index=sub['index'],
                    start=start,
                    end=end,
                    content=sub['text']
                )
            )
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(srt.compose(srt_subtitles))

    def translate_file(self, input_path: str, output_path: str):
        """ファイル全体を翻訳"""
        # SRTをJSONに変換
        subtitles = self.srt_to_json(input_path)
        
        # バッチ処理で翻訳
        translated_subtitles = []
        for i in tqdm(range(0, len(subtitles), self.batch_size), desc="Translating batches"):
            batch = subtitles[i:i + self.batch_size]
            try:
                translated_batch = self.translate_batch(batch)
                translated_subtitles.extend(translated_batch)
            except Exception as e:
                print(f"Error processing batch {i//self.batch_size + 1}: {str(e)}")
                translated_subtitles.extend(batch)  # エラーの場合は元のテキストを使用
                continue

        # 翻訳済みのJSONをSRTに変換
        self.json_to_srt(translated_subtitles, output_path)

    def add_line_breaks(self, text: str, max_line_length: int = 40, max_lines: int = 2) -> str:
        """テキストに適切な改行を追加"""
        tokenizer = Tokenizer()
        tokens = tokenizer.tokenize(text, wakati=True)
        lines = []
        current_line = ""

        for token in tokens:
            if re.match(r"[。！？]", token):
                current_line += token
                lines.append(current_line)
                current_line = ""
            elif len(current_line) + len(token) <= max_line_length:
                current_line += token
            else:
                lines.append(current_line)
                current_line = token

        if current_line:
            lines.append(current_line)

        lines = [re.sub(r'[$$$$［］「」$$（）\{\}｛｝''""]', '', line) for line in lines]
        return "\n".join(lines[:max_lines])

    def add_line_breaks_to_srt(self, srt_text: str) -> str:
        """SRTファイルの字幕データに改行を追加"""
        subtitles = list(srt.parse(srt_text))
        for subtitle in subtitles:
            subtitle.content = self.add_line_breaks(subtitle.content)
        return srt.compose(subtitles)


def main():
    if not GOOGLE_API_KEY:
        print("Error: GOOGLE_API_KEY environment variable is not set")
        return

    translator = SRTTranslator(api_key=GOOGLE_API_KEY, batch_size=20)

    input_path = select_srt_file()
    if input_path is None:
        return

    output_path = "output_translated.srt"
    translator.translate_file(input_path, output_path)

    # Add line breaks to SRT file
    with open(output_path, "r", encoding="utf-8") as f:
        srt_text = f.read()
    srt_text_with_line_breaks = translator.add_line_breaks_to_srt(srt_text)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(srt_text_with_line_breaks)

    # Create video with subtitles
    # video_file = input_path.replace(".srt", ".mp4")
    # output_video_file = video_file.replace(".mp4", "_jp.mp4")
    # create_video_with_subtitles(video_file, output_path, output_video_file)    

if __name__ == "__main__":
    main()